# Manufacturing Challenge 3 - Baseline Submission

This notebook provides a simple baseline for **Manufacturing Challenge 3: Production Delay Forecasting**.

**Goal**: Predict `HoursDelayed` for each shift-machine combination
**Metric**: Mean Squared Error (MSE) - Lower is better

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses only tabular shift workload data with a simple Random Forest regressor.


In [ ]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from agentds import BenchmarkClient

# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="your-api-key-here",        # Get from your team dashboard
    team_name="your-team-name-here"     # Your exact team name
)

# Load data from PVC paths
print("📂 Loading Manufacturing Challenge 3 data...")

# Load shift workload data
train_shifts = pd.read_csv("/home/jovyan/shared/datasets/Manufacturing/shift_workload_train.csv")
test_shifts = pd.read_csv("/home/jovyan/shared/datasets/Manufacturing/shift_workload_test.csv")

print(f"✅ Data loaded:")
print(f"   Train shifts: {train_shifts.shape}")
print(f"   Test shifts: {test_shifts.shape}")
print(f"   Train columns: {list(train_shifts.columns)}")
print(f"   Test columns: {list(test_shifts.columns)}")


In [ ]:
# 2. Tabular-Only Baseline Model and Predictions

# From data inspection - shift_workload columns:
# date, shift, machine_id, planned_jobs, avg_proc_time_min, std_proc_time_min, ambient_temp_c, HoursDelayed (train only)

# Select numeric shift features for baseline
shift_features = ['shift', 'planned_jobs', 'avg_proc_time_min', 'std_proc_time_min', 'ambient_temp_c']
print(f"📊 Using shift features: {shift_features}")

# Prepare training data
X_train = train_shifts[shift_features].fillna(0)
y_train = train_shifts['HoursDelayed']  # Target variable

# Prepare test data
X_test = test_shifts[shift_features].fillna(0)

# Train simple Random Forest baseline
print("🤖 Training Random Forest regressor...")
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Create submission file (format: date,shift,machine_id,HoursDelayed)
submission_df = pd.DataFrame({
    'date': test_shifts['date'],
    'shift': test_shifts['shift'],
    'machine_id': test_shifts['machine_id'],
    'HoursDelayed': predictions
})

# Save predictions
submission_df.to_csv("manufacturing_challenge3_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Delay range: {predictions.min():.2f} to {predictions.max():.2f} hours")


In [ ]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Manufacturing", 3, "manufacturing_challenge3_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating relevant information outside this table!")
print("   2. You've completed all Manufacturing challenges!")
